In [21]:
import numpy as np
import praw
import datetime
from nltk import tokenize, pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import requests
import boto3
import json

# bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, PanTool
from bokeh.plotting import figure, show

from CryptoSentWebApp import config, constants
from CryptoSentWebApp.objects import Comment

In [5]:
conf = config.get_secret('tst/CryptoSentWebApp/Reddit')

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
sid = SentimentIntensityAnalyzer()

In [8]:
reddit = praw.Reddit(
    client_id = conf['client_id'],
    client_secret = conf['client_secret'],
    username = conf['username'],
    password = conf['password'],
    user_agent = conf['user_agent']
)

# Obtain list of coins

In [10]:
coininfo_req = requests.get('https://www.cryptocompare.com/api/data/coinlist/').json()
coininfo = coininfo_req['Data']
coins = dict()
for item in coininfo.values():
    if int(item['SortOrder']) <= 5:
        coins.update({item['Name']: item['CoinName']})

In [11]:
coins

{'ETH': 'Ethereum',
 'BTC': 'Bitcoin',
 'XMR': 'Monero',
 'LTC': 'Litecoin',
 'DASH': 'Dash'}

# Test upstream

In [12]:
subreddit = reddit.subreddit('cryptocurrency')

In [13]:
kinesis = boto3.client('kinesis')

In [34]:
kinesis.list_streams()

{'StreamNames': ['CryptoUpStream'],
 'HasMoreStreams': False,
 'ResponseMetadata': {'RequestId': 'e4edf541-eff9-bc4a-b21c-8fc9ea9f8753',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e4edf541-eff9-bc4a-b21c-8fc9ea9f8753',
   'x-amz-id-2': 'CjYKC+bMQCzpoIc/DeHmJZtqjk4IEJJJn/qXLy0xS4OTTWhsUdV+owBs5NsWUjXMaKtKu/8KjEuWD5X+pTkA/SS5K4x0i7ES',
   'date': 'Sun, 22 Mar 2020 14:38:00 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '57'},
  'RetryAttempts': 0}}

In [33]:
[i for i in [{'test':1},  [{'test2': 2}, {'test3': ;3}]]]

SyntaxError: invalid syntax (<ipython-input-33-9328eb7f87ac>, line 1)

In [28]:
json.dumps([{1:2}, {3:4}])

'[{"1": 2}, {"3": 4}]'

In [24]:
for comment in subreddit.stream.comments():
    # Extract data on comment level
    comment_id = comment.id
    comment_author = comment.author
    comment_link = comment.link_id
    comment_body = comment.body
    comment_ts = datetime.datetime.utcfromtimestamp(comment.created_utc)
    print(f'Id:{comment_id} Author:{comment_author} Submission:{comment_link}\nComment: {comment_body}\n')
    comment_sentences = tokenize.sent_tokenize(comment_body)
    
    # Extract data on sentence level
    for sentence in comment_sentences:    
        senti = sid.polarity_scores(sentence)
        words = tokenize.word_tokenize(sentence)
        subject = set()
        for word in words:
            if word in coins.keys() or word.upper() in coins.keys():
                subject.add(coins[word.upper()])
            elif word in coins.values() or word.capitalize() in coins.values():
                subject.add(word.capitalize())
        print(f'{comment_ts}: subject:{subject} {sentence} [{senti}]\n')
    print('\n')

Id:fl6c5ib Author:parakite Submission:t3_fmq6bx
Comment: Never heard of Skale. Let me check what it is. Or if it still exists. You never know with these one/few engineer startups.

2020-03-22 07:31:28: subject:set() Never heard of Skale. [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]

2020-03-22 07:31:28: subject:set() Let me check what it is. [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]

2020-03-22 07:31:28: subject:set() Or if it still exists. [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]

2020-03-22 07:31:28: subject:set() You never know with these one/few engineer startups. [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]



Id:fl6cje9 Author:paparaven Submission:t3_fmq6bx
Comment: You put in 100 on a 5x leverage and everytime the price moves by 10% you lose/gain 50 instead of 10. I wouldn’t recommend doing this with crypto, you’ll probably end up homeless.

2020-03-22 07:39:12: subject:set() You put in 100 on a 5x leverage and everytime the pri

KeyboardInterrupt: 

In [ ]:
        # Fetch the list of coins and create lists with all their names and tickersymbols
        CoinNames = coinlist.CoinName.values
        LowerCoinNames = coinlist.CoinName.str.lower().values
        Names = coinlist.Name.values

        CoinNamesSplit, CoinNamesSplitLower, Splitted = [], [], []
        for coin in CoinNames:
            if ' ' in coin:
                Splitted += [coin]
                tagged = nltk.pos_tag(nltk.word_tokenize(coin))
                for word in tagged:
                    if word[1].startswith('NN') and (word[0].isalpha() or word[0].isnumeric()) and word[0] not in Names:
                        CoinNamesSplit += [word[0]]
                        CoinNamesSplitLower += [word[0].lower()]

        # Put names that occur multiple times on a blacklist so duplicates cannot occur
        CountWords = Counter(CoinNamesSplit)
        CoinBlacklist = []
        for word in CountWords.most_common():
            if word[1] > 1:
                CoinBlacklist += [word[0]]

        # Remove special characters from coin abbreviations before identifying names within posts and comments
        for i, name in enumerate(Names):
            if '*' in name:
                Names[i] = re.sub(r'\W+', '', name)

        post_coins = [{'CoinName': set(),
                       'Name': set(),
                       'LowerCoinName': set(),
                       'CoinNamesSplit': set(),
                       'LowerLowerCoinName': set()} for _ in range(len(CryptoComm['post_tagged']))]
        for i, post in enumerate(CryptoComm['post_tagged']):
            for word in post:
                if word[0] in CoinNames:
                    post_coins[i]['CoinName'].add(word[0])
                if word[0] in LowerCoinNames:
                    post_coins[i]['LowerCoinName'].add(word[0])
                if word[0] in Names:
                    post_coins[i]['Name'].add(word[0])
                if word[0] in CoinNamesSplit and word[1].startswith('NN') and word[0] not in CoinBlacklist:
                    post_coins[i]['CoinNamesSplit'].add(word[0])

        CryptoComm['PostCoinMatches'] = post_coins

        comment_coins = [{'CoinName': set(),
                          'Name': set(),
                          'LowerCoinName': set(),
                          'CoinNamesSplit': set(),
                          'LowerCoinNamesSplit': set()} for _ in range(len(CryptoComm['comment_tagged']))]
        for i, comment in enumerate(CryptoComm['comment_tagged']):
            for word in comment:
                if word[0] in CoinNames:
                    comment_coins[i]['CoinName'].add(word[0])
                if word[0] in LowerCoinNames:
                    comment_coins[i]['LowerCoinName'].add(word[0])
                if word[0] in Names:
                    comment_coins[i]['Name'].add(word[0])
                if word[0] in CoinNamesSplit and word[1].startswith('NN') and word[0] not in CoinBlacklist:
                    comment_coins[i]['CoinNamesSplit'].add(word[0])
                if (word[0] in CoinNamesSplitLower and word[1].startswith('NN')
                        and word[0].capitalize() not in CoinBlacklist):
                    comment_coins[i]['LowerCoinNamesSplit'].add(word[0])


In [8]:
def moving_average(window, t=10, start=0):
    if len(window) < t: 
        return [start]
    return np.convolve(window[-t:], np.ones(t, dtype=float), mode="valid") / t

In [124]:
def subreddit_sentiment(subreddit):
    ma_data = ColumnDataSource(dict(timestamp=[], sentiment_ma=[]))    
    sentence_data = ColumnDataSource(dict(timestamp=[], sentiment=[]))
    hover = HoverTool(
        tooltips=[('time', '@timestamp{%Y-%m-%d %H:%M:%S}'), ('sentiment', '@sentiment_ma{%0.3f}')],
        formatters={'timestamp': 'datetime', 'sentiment_ma': 'printf'},
        mode='vline'
    )
    p = figure(
        plot_width=800,
        plot_height=400,
        x_axis_type='datetime',
        tools=[hover, PanTool(dimensions='height')],
        title="Real-Time sentiment plot",
        y_range=(-1, 1)
    )
    dots = p.circle(source=sentence_data, x='timestamp', y='sentiment')
    lin = p.line(source=ma_data, x='timestamp', y='sentiment_ma', line_color='red', line_width=2)
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Reddit comment sentiment score"

    target = show(p, notebook_handle=True)
    subreddit = reddit.subreddit(subreddit)
    total_sentiment = []
    for comment in subreddit.stream.comments():
        comment_sent = tokenize.sent_tokenize(comment.body)
        sentence_sentiment = []
        for sentence in comment_sent:
            # Extract the sentiment from the sentence
            sentence_sentiment.append(sid.polarity_scores(sentence)['compound'])
            
        avg_sentence_sentiment = sum(sentence_sentiment) / len(sentence_sentiment)
        total_sentiment.append(avg_sentence_sentiment)
        sentence_data.stream(dict(
            timestamp=[datetime.datetime.utcfromtimestamp(comment.created_utc)],
            sentiment=[avg_sentence_sentiment]
        ), 1000)
        push_notebook(handle=target)    
        
        if len(total_sentiment) > 100:
            ma_data.stream(dict(
                timestamp=[datetime.datetime.utcfromtimestamp(comment.created_utc)],
                sentiment_ma=[moving_average(sentence_data.data['sentiment'], 100)]
            ), 11)
            total_sentiment=[]
            push_notebook(handle=target)

In [125]:
subreddit_sentiment('all')

KeyboardInterrupt: 

In [11]:
sid = SentimentIntensityAnalyzer()
for submission in subreddit.stream.submissions():
    sent = sid.polarity_scores(submission.title)
    print(submission.title, sent)

NameError: name 'subreddit' is not defined